#   机器学习毕业项目
##   项目: Dogs vs. Cats Redux: Kernels Edition
###   在这个项目中, 我们将通过评估几种模型与不同程度的调优, 来不断优化我们的模型框架, 以达到我们最终的top 10%标准.
---
###   写在前面
-   这次的毕业项目选做猫狗, 一方面是因为资料比较多, 资源比较丰富. 另外一方面, 图形识别也是当下的一大热点. 虽然毕业后可能会继续从事传统机器学习方面的研究, 但是能够有这段经历还是很不错的, 也能丰富自己的简历与深度学习的知识.

###   关于DataSet
-   本文中的DataSet是已经从kaggle上拖取到本地, 并放入当前工作目录中, 但是由于Github的大小限制, 如果你想要执行并研究其中的一些代码, 请自行准备好数据集, 并存放到./DataSet目录下. kaggle传送门 <a href='https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data'>dogs-vs-cats-redux-kernels-edition/data</a>.

Dogs_vs_Cats.ipynb  README.md           proposal.pdf
